# A/B Test: 이미지 생성 프롬프트 비교

## 목적
- **대조군 A**: 일반적인 OpenAI API를 활용해 단순히 생성한 이미지 생성용 영어 프롬프트
- **실험군 B**: 업종별 분류(21개 하위 그룹)를 통해 특화된 이미지 생성용 영어 프롬프트

## 가설
업종별 특화 프롬프트 시스템(B)이 일반 프롬프트 생성(A)보다 더 구체적이고 업종 특성에 맞는 프롬프트를 생성한다.

## 테스트 구성
- 21개 하위 그룹별 10개의 한글 예시 문장
- 총 210개의 테스트 케이스
- 4개의 셀로 구성: S등급, A등급, B+C등급, D+E등급

## 0. 환경 설정 및 모듈 로드

In [1]:
import os
import sys
import json
from typing import Dict, List
from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 루트 경로 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 환경변수 로드
load_dotenv()

# 실험군 B용 모듈 import
from src.generation.image_generation.prompt.prompt_manager import PromptTemplateManager

print(f"프로젝트 경로: {PROJECT_ROOT}")
print("모듈 로드 완료!")

프로젝트 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz
모듈 로드 완료!


## 1. A/B 테스트 함수 정의

In [2]:
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 실험군 B용 프롬프트 매니저 초기화
prompt_manager_b = PromptTemplateManager()


def generate_prompt_control_a(korean_input: str) -> str:
    """
    대조군 A: 일반적인 OpenAI API를 사용한 프롬프트 생성
    
    특징:
    - 업종 분류 없이 단순 번역 및 프롬프트 변환
    - 일반적인 이미지 생성 지시문 사용
    - 실험군 B와 비슷한 길이(80-250 단어)로 생성
    """
    system_prompt = """You are an image prompt generator. Convert Korean advertising text into English image generation prompts.

Rules:
1. Create a detailed prompt (80-250 words) for AI image generation
2. Include subject, setting, lighting, atmosphere, and visual details
3. Write in natural flowing sentences
4. Focus on visual elements that can be photographed or illustrated
5. Do NOT include any text or typography in the image description

Output format:
{"positive": "your detailed prompt here"}
"""
    
    user_prompt = f"""Convert this Korean advertising text into an image generation prompt:

{korean_input}

Generate a detailed, visually descriptive prompt."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7
        )
        
        result = response.choices[0].message.content.strip()
        
        # JSON 파싱
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()
        
        data = json.loads(result)
        return data.get("positive", result)
        
    except Exception as e:
        return f"[Error] {str(e)}"


def generate_prompt_experiment_b(korean_input: str, style: str = "realistic") -> str:
    """
    실험군 B: 업종별 분류를 통한 특화 프롬프트 생성
    
    특징:
    - 한글 입력에서 자동 업종 감지
    - 18개 하위 그룹별 최적화된 프롬프트 템플릿 적용
    - 업종별 키워드, 조명, 구도, 색감 등 반영
    """
    try:
        result = prompt_manager_b.generate_detailed_prompt(korean_input, style)
        return result.get("positive", "")
    except Exception as e:
        return f"[Error] {str(e)}"


def run_ab_test(test_cases: List[Dict], group_name: str):
    """
    A/B 테스트 실행 및 결과 출력
    
    Args:
        test_cases: 테스트 케이스 리스트 [{"group": "S-1", "input": "한글 입력"}]
        group_name: 그룹명 (예: "S등급")
    """
    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 결과")
    print(f"{'='*100}\n")
    
    results = []
    
    for i, case in enumerate(test_cases, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")
        
        print(f"\n[{i}/{len(test_cases)}] {group}")
        print(f"-" * 80)
        
        # A/B 프롬프트 생성
        prompt_a = generate_prompt_control_a(korean_input)
        prompt_b = generate_prompt_experiment_b(korean_input)
        
        # 결과 출력
        print(f'한글 입력: "{korean_input}"')
        print(f'\n대조군 A: "{prompt_a[:500]}..."' if len(prompt_a) > 500 else f'\n대조군 A: "{prompt_a}"')
        print(f'\n실험군 B: "{prompt_b[:500]}..."' if len(prompt_b) > 500 else f'\n실험군 B: "{prompt_b}"')
        print(f"\n📏 길이 비교: A={len(prompt_a.split())}단어, B={len(prompt_b.split())}단어")
        
        results.append({
            "group": group,
            "korean_input": korean_input,
            "prompt_a": prompt_a,
            "prompt_b": prompt_b,
            "len_a": len(prompt_a.split()),
            "len_b": len(prompt_b.split())
        })
    
    # 요약 통계
    avg_len_a = sum(r["len_a"] for r in results) / len(results)
    avg_len_b = sum(r["len_b"] for r in results) / len(results)
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 요약 통계")
    print(f"{'='*100}")
    print(f"테스트 케이스 수: {len(results)}")
    print(f"대조군 A 평균 길이: {avg_len_a:.1f} 단어")
    print(f"실험군 B 평균 길이: {avg_len_b:.1f} 단어")
    
    return results

## 2. 테스트 데이터 정의

### 21개 하위 그룹별 10개의 한글 예시 문장

In [3]:
# ============================================
# S등급: 감성 및 미각 자극 그룹 (6개 그룹)
# ============================================

# S-1: 고온 조리 및 육즙 강조 (17개 업종)
S1_HOT_COOKING = [
    {"group": "S-1", "input": "숯불에 구운 삼겹살, 육즙이 뚝뚝 떨어지는 모습"},
    {"group": "S-1", "input": "치킨 전문점 홍보, 갓 튀긴 바삭한 치킨"},
    {"group": "S-1", "input": "피자 가게 신메뉴, 치즈가 늘어나는 장면"},
    {"group": "S-1", "input": "마라탕 전문점, 얼얼하게 매운 국물과 재료들"},
    {"group": "S-1", "input": "스테이크 레스토랑, 미디엄 레어로 구운 안심"},
    {"group": "S-1", "input": "중국집 짜장면, 윤기 나는 면발과 춘장 소스"},
    {"group": "S-1", "input": "버거 전문점, 육즙 가득한 수제버거"},
    {"group": "S-1", "input": "곱창 구이, 불판 위에서 지글지글 익는 모습"},
    {"group": "S-1", "input": "일식 라멘집, 김이 모락모락 나는 돈코츠 라멘"},
    {"group": "S-1", "input": "베트남 쌀국수, 허브와 숙주가 곁들여진 뜨끈한 포"},
]

# S-2: 신선도 및 색감 강조 (8개 업종)
S2_FRESHNESS = [
    {"group": "S-2", "input": "횟집 홍보, 신선한 모둠회와 싱싱한 해산물"},
    {"group": "S-2", "input": "초밥 전문점, 장인이 만든 프리미엄 오마카세"},
    {"group": "S-2", "input": "해산물 찜, 게와 새우가 가득한 해물찜"},
    {"group": "S-2", "input": "수산시장 홍보, 싱싱한 활어와 조개류"},
    {"group": "S-2", "input": "복어 요리 전문점, 복어회와 복어탕 코스"},
    {"group": "S-2", "input": "정육점 홍보, 마블링이 좋은 한우 투플러스"},
    {"group": "S-2", "input": "과일 전문점, 제철 딸기와 샤인머스캣"},
    {"group": "S-2", "input": "채소 가게, 유기농 신선 채소 모음"},
    {"group": "S-2", "input": "연어 전문점, 노르웨이산 생연어 사시미"},
    {"group": "S-2", "input": "젓갈 가게, 전통 방식으로 담근 명란젓"},
]

# S-3: 감성 및 부드러운 질감 (11개 업종)
S3_EMOTIONAL = [
    {"group": "S-3", "input": "카페 신메뉴, 딸기 라떼와 크루아상"},
    {"group": "S-3", "input": "베이커리 홍보, 갓 구운 소금빵과 크림빵"},
    {"group": "S-3", "input": "아이스크림 가게, 파스텔 톤 젤라또 3종"},
    {"group": "S-3", "input": "떡카페, 예쁜 화과자와 전통차 세트"},
    {"group": "S-3", "input": "샌드위치 전문점, 건강한 샐러드 샌드위치"},
    {"group": "S-3", "input": "슈퍼마켓 홍보, 신선식품 코너 할인 행사"},
    {"group": "S-3", "input": "편의점 간편식, 도시락과 샐러드 신제품"},
    {"group": "S-3", "input": "우유 전문점, 신선한 목장 우유 배달"},
    {"group": "S-3", "input": "아이스크림 할인점, 인기 아이스크림 반값 세일"},
    {"group": "S-3", "input": "생수/음료 배달, 시원한 여름 음료 패키지"},
]

# S-4: 정갈한 한식과 상차림 (9개 업종)
S4_KOREAN_HOMESTYLE = [
    {"group": "S-4", "input": "한정식 홍보, 정갈한 반찬과 된장찌개 상차림"},
    {"group": "S-4", "input": "뚝배기 국밥집, 뜨끈한 돼지국밥 한 그릇"},
    {"group": "S-4", "input": "족발/보쌈 전문점, 쫄깃한 족발과 보쌈"},
    {"group": "S-4", "input": "전문점 홍보, 바삭한 해물파전과 막걸리"},
    {"group": "S-4", "input": "칼국수집, 손칼국수와 왕만두"},
    {"group": "S-4", "input": "냉면 전문점, 시원한 물냉면과 비빔냉면"},
    {"group": "S-4", "input": "김밥 전문점, 다양한 김밥과 라면 세트"},
    {"group": "S-4", "input": "백반집 점심 메뉴, 오늘의 집밥 정식"},
    {"group": "S-4", "input": "반찬 가게, 정성 가득 수제 반찬 10종"},
    {"group": "S-4", "input": "분식집, 떡볶이와 순대 세트 메뉴"},
]

# S-5: 글로벌 다채로움 (4개 업종)
S5_GLOBAL_VARIETY = [
    {"group": "S-5", "input": "기타 일식 음식점, 오코노미야끼와 타코야끼"},
    {"group": "S-5", "input": "기타 서양식 레스토랑, 리조또와 뇨끼 세트"},
    {"group": "S-5", "input": "동남아 전문점, 팟타이와 쏨땀 세트"},
    {"group": "S-5", "input": "퓨전 아시안 레스토랑, 동양과 서양의 조화"},
    {"group": "S-5", "input": "태국 음식점, 향긋한 그린커리와 똠얌꿍"},
    {"group": "S-5", "input": "인도 커리 전문, 탄두리 치킨과 난"},
    {"group": "S-5", "input": "멕시칸 레스토랑, 화려한 색감의 타코와 부리또"},
    {"group": "S-5", "input": "지중해 레스토랑, 올리브유와 허브 가득한 요리"},
    {"group": "S-5", "input": "분류 안된 외국식, 다국적 퓨전 요리 한 상"},
    {"group": "S-5", "input": "스페인 타파스, 다채로운 소접시 요리들"},
]

# S-6: 화려한 무드와 야간 감성 (4개 업종)
S6_NIGHTLIFE = [
    {"group": "S-6", "input": "생맥주 전문점, 네온 불빛 아래 시원한 맥주"},
    {"group": "S-6", "input": "요리주점 인테리어, 따뜻한 조명과 안주"},
    {"group": "S-6", "input": "일반 유흥 주점, 루프탑 바 야경과 칵테일"},
    {"group": "S-6", "input": "무도 유흥 주점, 화려한 조명과 춤추는 실루엣"},
    {"group": "S-6", "input": "이자카야 분위기, 일본식 등롱과 나무 인테리어"},
    {"group": "S-6", "input": "와인바 디스플레이, 와인 셀러와 보케 조명"},
    {"group": "S-6", "input": "크래프트 비어펍, 탭 핸들과 호프 장식"},
    {"group": "S-6", "input": "재즈바 무대, 악기와 무드등 연출"},
    {"group": "S-6", "input": "칵테일 라운지, 바텐더의 믹솔로지 퍼포먼스"},
    {"group": "S-6", "input": "포장마차 감성, 빨간 조명과 어묵탕"},
]

# S등급 전체 테스트 케이스
S_GRADE_TEST_CASES = S1_HOT_COOKING + S2_FRESHNESS + S3_EMOTIONAL + S4_KOREAN_HOMESTYLE + S5_GLOBAL_VARIETY + S6_NIGHTLIFE

print(f"S등급 테스트 케이스 수: {len(S_GRADE_TEST_CASES)}개")

S등급 테스트 케이스 수: 60개


In [4]:
# ============================================
# A등급: 비주얼 변화 및 자기관리 그룹 (4개 그룹)
# ============================================

# A-1: 인물 중심 뷰티 및 에스테틱 (6개 업종)
A1_BEAUTY = [
    {"group": "A-1", "input": "미용실 봄 헤어 스타일, 웨이브 펌 시술 결과"},
    {"group": "A-1", "input": "피부관리실 홍보, 맑고 깨끗한 피부 관리"},
    {"group": "A-1", "input": "네일샵 신상 디자인, 봄 꽃 네일 아트"},
    {"group": "A-1", "input": "성형외과 상담 안내, 자연스러운 코 성형"},
    {"group": "A-1", "input": "피부과 레이저 시술, 잡티 제거 전후 비교"},
    {"group": "A-1", "input": "미용실 염색 이벤트, 트렌디한 발레아주"},
    {"group": "A-1", "input": "에스테틱 페이셜 케어, 광채 나는 피부"},
    {"group": "A-1", "input": "네일샵 젤네일, 고급스러운 글리터 디자인"},
    {"group": "A-1", "input": "화장품 매장, 봄 시즌 신제품 립스틱"},
    {"group": "A-1", "input": "속눈썹 연장 전문, 자연스러운 볼륨 래쉬"},
]

# A-2: 신체 변화 및 웰니스 (6개 업종)
A2_WELLNESS = [
    {"group": "A-2", "input": "헬스장 PT 프로그램, 1:1 개인 트레이닝"},
    {"group": "A-2", "input": "요가 스튜디오, 아침 명상 요가 클래스"},
    {"group": "A-2", "input": "필라테스 센터, 체형 교정 프로그램"},
    {"group": "A-2", "input": "태권도 도장, 어린이 태권도 수업"},
    {"group": "A-2", "input": "종합 스포츠시설, 다양한 운동 프로그램"},
    {"group": "A-2", "input": "체형/비만 관리, 3개월 체중 감량 성공"},
    {"group": "A-2", "input": "마사지샵 홍보, 타이 마사지 전신 케어"},
    {"group": "A-2", "input": "크로스핏 박스, 고강도 인터벌 트레이닝"},
    {"group": "A-2", "input": "스피닝 센터, 신나는 그룹 사이클링"},
    {"group": "A-2", "input": "무술학원, 주짓수와 복싱 레슨"},
]

# A-3: 패션 및 스타일 아이템 (18개 업종)
A3_FASHION = [
    {"group": "A-3", "input": "여성 의류 매장, 봄 신상 원피스 컬렉션"},
    {"group": "A-3", "input": "남성 정장 전문점, 맞춤 수트 제작"},
    {"group": "A-3", "input": "아동복 가게, 귀여운 봄 키즈 패션"},
    {"group": "A-3", "input": "신발 매장, 트렌디한 스니커즈 신상"},
    {"group": "A-3", "input": "가방 브랜드, 가죽 토트백 신제품"},
    {"group": "A-3", "input": "인테리어 디자인, 모던 거실 인테리어"},
    {"group": "A-3", "input": "시각 디자인업, 브랜드 로고 디자인"},
    {"group": "A-3", "input": "패션 디자인, 시즌 컬렉션 런웨이"},
    {"group": "A-3", "input": "웨딩샵 드레스, 우아한 레이스 웨딩드레스"},
    {"group": "A-3", "input": "한복 대여점, 설날 전통 한복 렌탈"},
]

# A-4: 섬세한 케어 및 감성 소품 (11개 업종)
A4_DELICATE_CARE = [
    {"group": "A-4", "input": "꽃집 홍보, 발렌타인 장미 꽃다발"},
    {"group": "A-4", "input": "반려동물 용품점, 강아지 봄 옷 신상"},
    {"group": "A-4", "input": "동물병원 안내, 반려견 건강검진 패키지"},
    {"group": "A-4", "input": "가구점 홍보, 미니멀 원목 책상 세트"},
    {"group": "A-4", "input": "주방용품점, 프리미엄 조리도구 세트"},
    {"group": "A-4", "input": "건강보조식품, 비타민과 유산균 패키지"},
    {"group": "A-4", "input": "액세서리 매장, 봄 시즌 주얼리 컬렉션"},
    {"group": "A-4", "input": "시계/귀금속, 프리미엄 시계 컬렉션"},
    {"group": "A-4", "input": "안경점 신상, 트렌드 뿔테 안경"},
    {"group": "A-4", "input": "사진관 홍보, 가족 기념사진 촬영"},
]

# A등급 전체 테스트 케이스
A_GRADE_TEST_CASES = A1_BEAUTY + A2_WELLNESS + A3_FASHION + A4_DELICATE_CARE

print(f"A등급 테스트 케이스 수: {len(A_GRADE_TEST_CASES)}개")

A등급 테스트 케이스 수: 40개


In [5]:
# ============================================
# B등급: 공간 체험 및 교육 그룹 (3개 그룹)
# ============================================

# B-1: 학습 및 몰입 (18개 업종)
B1_LEARNING = [
    {"group": "B-1", "input": "입시학원 홍보, 소수정예 수학 클래스"},
    {"group": "B-1", "input": "외국어 학원, 원어민 영어 회화 수업"},
    {"group": "B-1", "input": "코딩 학원, 어린이 프로그래밍 교육"},
    {"group": "B-1", "input": "스터디카페 홍보, 조용하고 쾌적한 학습 공간"},
    {"group": "B-1", "input": "운전학원 안내, 친절한 1:1 운전 교육"},
    {"group": "B-1", "input": "공무원 학원, 합격자 수기와 커리큘럼"},
    {"group": "B-1", "input": "독서실 오픈, 24시간 개인 학습 부스"},
    {"group": "B-1", "input": "음악학원, 피아노와 기타 레슨"},
    {"group": "B-1", "input": "미술학원 홍보, 수채화 클래스 작품전"},
    {"group": "B-1", "input": "만화방, 아늑한 독서 공간과 신간 만화"},
]

# B-2: 여행 및 숙박 (9개 업종)
B2_LEISURE = [
    {"group": "B-2", "input": "호텔 프로모션, 오션뷰 스위트룸 패키지"},
    {"group": "B-2", "input": "펜션 홍보, 가족 단위 바베큐 독채"},
    {"group": "B-2", "input": "글램핑장 오픈, 자연 속 럭셔리 캠핑"},
    {"group": "B-2", "input": "여행사 패키지, 제주도 3박 4일 투어"},
    {"group": "B-2", "input": "목욕탕/사우나, 힐링 스파 시설 안내"},
    {"group": "B-2", "input": "여관/모텔, 깔끔한 비즈니스 숙박"},
    {"group": "B-2", "input": "캠핑장, 오토캠핑 사이트 예약 안내"},
    {"group": "B-2", "input": "고시원, 1인실 원룸형 주거 공간"},
    {"group": "B-2", "input": "리조트 스파, 온천과 풀빌라 패키지"},
    {"group": "B-2", "input": "기타 여행 서비스, 투어 가이드 예약"},
]

# B-3: 스포츠 및 활동 (19개 업종)
B3_SPORTS = [
    {"group": "B-3", "input": "수영장 회원 모집, 성인 수영 강습"},
    {"group": "B-3", "input": "골프 연습장, 실내 스크린 골프 시설"},
    {"group": "B-3", "input": "볼링장 이벤트, 단체 볼링 대회"},
    {"group": "B-3", "input": "당구장 홍보, 프로 당구 레슨"},
    {"group": "B-3", "input": "테니스장 오픈, 실내 테니스 코트"},
    {"group": "B-3", "input": "탁구장 클럽, 동호인 리그전"},
    {"group": "B-3", "input": "노래방, 최신 음향 시스템과 넓은 룸"},
    {"group": "B-3", "input": "PC방, RGB 조명과 고사양 게이밍 PC"},
    {"group": "B-3", "input": "전자 게임장, 복고풍 아케이드 게임"},
    {"group": "B-3", "input": "낚시터, 자연 속 힐링 낚시 체험"},
]

# ============================================
# C등급: 신뢰 기반 전문 서비스 그룹 (3개 그룹)
# ============================================

# C-1: 전문 사무 (26개 업종)
C1_PROFESSIONAL = [
    {"group": "C-1", "input": "부동산 중개소, 역세권 신축 오피스텔 매물"},
    {"group": "C-1", "input": "법무법인 홍보, 기업 법률 자문 서비스"},
    {"group": "C-1", "input": "세무사 사무소, 종합소득세 신고 대행"},
    {"group": "C-1", "input": "회계사 상담, 스타트업 재무 컨설팅"},
    {"group": "C-1", "input": "특허 사무소, 상표권 등록 안내"},
    {"group": "C-1", "input": "노무사 사무실, 근로계약서 검토 서비스"},
    {"group": "C-1", "input": "광고 대행사, 브랜드 마케팅 캠페인"},
    {"group": "C-1", "input": "경영 컨설팅, 중소기업 성장 전략"},
    {"group": "C-1", "input": "행정사 사무소, 인허가 대행 서비스"},
    {"group": "C-1", "input": "번역 사무소, 법률 문서 번역 전문"},
]

# C-2: 의료 및 케어 (18개 업종)
C2_MEDICAL = [
    {"group": "C-2", "input": "내과 의원, 건강검진 패키지 안내"},
    {"group": "C-2", "input": "치과 클리닉, 임플란트 상담 이벤트"},
    {"group": "C-2", "input": "한의원 홍보, 비만 한방 다이어트"},
    {"group": "C-2", "input": "정형외과, 척추 비수술 치료 프로그램"},
    {"group": "C-2", "input": "안과 라식 센터, 시력교정술 상담"},
    {"group": "C-2", "input": "이비인후과, 수면무호흡 검사 안내"},
    {"group": "C-2", "input": "소아과 의원, 영유아 예방접종"},
    {"group": "C-2", "input": "산부인과, 임신 초기 검사 프로그램"},
    {"group": "C-2", "input": "약국 홍보, 건강기능식품 상담"},
    {"group": "C-2", "input": "요양병원 안내, 어르신 전문 케어 시설"},
]

# C-3: 기술 및 시공 (13개 업종)
C3_TECHNICAL = [
    {"group": "C-3", "input": "자동차 정비소, 엔진오일 교환 이벤트"},
    {"group": "C-3", "input": "세차장 홍보, 프리미엄 손세차 서비스"},
    {"group": "C-3", "input": "모터사이클 수리, 오토바이 정비 전문"},
    {"group": "C-3", "input": "건축 설계 사무소, 주택 설계 컨설팅"},
    {"group": "C-3", "input": "조경 설계, 정원 디자인 시공"},
    {"group": "C-3", "input": "에어컨 청소, 분해 완전 세척 서비스"},
    {"group": "C-3", "input": "방역 소독 업체, 바퀴벌레 퇴치 서비스"},
    {"group": "C-3", "input": "건물 청소 서비스, 입주 청소 전문"},
    {"group": "C-3", "input": "시설 유지관리, 아파트 관리 대행"},
    {"group": "C-3", "input": "도배/장판 업체, 벽지 시공 견적 안내"},
]

# B+C등급 전체 테스트 케이스
BC_GRADE_TEST_CASES = B1_LEARNING + B2_LEISURE + B3_SPORTS + C1_PROFESSIONAL + C2_MEDICAL + C3_TECHNICAL

print(f"B+C등급 테스트 케이스 수: {len(BC_GRADE_TEST_CASES)}개")

B+C등급 테스트 케이스 수: 60개


In [6]:
# ============================================
# D등급: 목적형 소매 및 장비 그룹 (3개 그룹)
# ============================================

# D-1: 중장비 및 부품 (13개 업종)
D1_HEAVY_EQUIPMENT = [
    {"group": "D-1", "input": "타이어 전문점, 사계절 타이어 할인"},
    {"group": "D-1", "input": "자동차 부품점, 순정 브레이크 패드"},
    {"group": "D-1", "input": "철물점 홍보, 공구 세트 특가 판매"},
    {"group": "D-1", "input": "건축자재 판매, 시멘트/벽돌 도매"},
    {"group": "D-1", "input": "전기용품 가게, LED 조명 설치"},
    {"group": "D-1", "input": "중장비 대여, 굴삭기 렌탈 서비스"},
    {"group": "D-1", "input": "바닥재 전문, 마루/장판 시공"},
    {"group": "D-1", "input": "모터사이클 부품, 오토바이 액세서리"},
    {"group": "D-1", "input": "가정용 연료 판매, 등유 배달 서비스"},
    {"group": "D-1", "input": "포장/충전 서비스, 산업용 가스 충전"},
]

# D-2: 일반 기기 (14개 업종)
D2_GENERAL_EQUIPMENT = [
    {"group": "D-2", "input": "핸드폰 매장, 최신 스마트폰 개통"},
    {"group": "D-2", "input": "컴퓨터 수리점, 노트북 SSD 업그레이드"},
    {"group": "D-2", "input": "가전제품 매장, 에어컨 여름 특가"},
    {"group": "D-2", "input": "복합기 임대, 사무실 프린터 렌탈"},
    {"group": "D-2", "input": "폰 수리 전문, 액정 교체 당일 수리"},
    {"group": "D-2", "input": "렌터카 업체, 장기 렌트 프로모션"},
    {"group": "D-2", "input": "세탁소, 드라이클리닝 할인 이벤트"},
    {"group": "D-2", "input": "셀프 빨래방, 24시간 코인 세탁"},
    {"group": "D-2", "input": "중고 상품점, 가전제품 매입/판매"},
    {"group": "D-2", "input": "TV 수리점, 대형 TV 출장 서비스"},
]

# D-3: 취미 및 라이프스타일 소매 (18개 업종)
D3_HOBBY_LIFESTYLE = [
    {"group": "D-3", "input": "악기점 진열, 통기타와 일렉기타 컬렉션"},
    {"group": "D-3", "input": "LP 레코드샵, 빈티지 턴테이블과 앨범들"},
    {"group": "D-3", "input": "전자담배 전문점, 다양한 기기와 액상 진열"},
    {"group": "D-3", "input": "와인샵 셀러, 정렬된 와인 병과 목재 선반"},
    {"group": "D-3", "input": "서점, 아늑한 책장과 베스트셀러"},
    {"group": "D-3", "input": "문구점 진열대, 다이어리와 펜 컬렉션"},
    {"group": "D-3", "input": "장난감 가게, 레고와 인형 신제품"},
    {"group": "D-3", "input": "운동용품점, 덤벨과 요가매트 진열"},
    {"group": "D-3", "input": "자전거 매장, 로드바이크와 MTB 전시"},
    {"group": "D-3", "input": "기념품샵, 마그넷과 엽서 컬렉션"},
]

# ============================================
# E등급: 인프라 및 특수 목적 그룹 (2개 그룹)
# ============================================

# E-1: 편의 및 인프라 (3개 업종)
E1_INFRASTRUCTURE = [
    {"group": "E-1", "input": "주유소 홍보, 리터당 할인 이벤트"},
    {"group": "E-1", "input": "LPG 충전소, 자동차 가스 충전"},
    {"group": "E-1", "input": "셀프 주유소, 24시간 야간 운영"},
    {"group": "E-1", "input": "세차장 겸 주유소, 셀프 세차 시설"},
    {"group": "E-1", "input": "전기차 충전소, 급속 충전 스테이션"},
    {"group": "E-1", "input": "휴게소 주유소, 깔끔한 시설 전경"},
    {"group": "E-1", "input": "야간 주유소, 불빛 아래 차량 급유"},
    {"group": "E-1", "input": "카페 겸 주유소, 모던한 휴게 공간"},
    {"group": "E-1", "input": "수소 충전소, 친환경 인프라 설비"},
    {"group": "E-1", "input": "얼음 소매, 여름 대용량 얼음 배달"},
]

# E-2: 예절 및 의례 (2개 업종)
E2_CEREMONY = [
    {"group": "E-2", "input": "장례식장 안내, 엄숙한 추모 공간"},
    {"group": "E-2", "input": "납골당 분양, 평화로운 영면 장소"},
    {"group": "E-2", "input": "상조 서비스, 장례 종합 패키지"},
    {"group": "E-2", "input": "수목장 안내, 자연과 함께하는 추모"},
    {"group": "E-2", "input": "추모관 홍보, 차분한 추모 시설"},
    {"group": "E-2", "input": "장례 화환 배치, 근조 꽃 장식"},
    {"group": "E-2", "input": "화장터 안내, 정갈한 추모 절차"},
    {"group": "E-2", "input": "장지 전경, 고요한 자연 풍경"},
    {"group": "E-2", "input": "추모 기념관 로비, 따뜻한 조명"},
    {"group": "E-2", "input": "예배당 내부, 평화로운 영성 공간"},
]

# D+E등급 전체 테스트 케이스
DE_GRADE_TEST_CASES = D1_HEAVY_EQUIPMENT + D2_GENERAL_EQUIPMENT + D3_HOBBY_LIFESTYLE + E1_INFRASTRUCTURE + E2_CEREMONY

print(f"D+E등급 테스트 케이스 수: {len(DE_GRADE_TEST_CASES)}개")

D+E등급 테스트 케이스 수: 50개


## 3. A/B 테스트 실행

### 3.1 S등급 테스트 (감성 및 미각 자극)

In [7]:
# S등급 A/B 테스트 실행
s_grade_results = run_ab_test(S_GRADE_TEST_CASES, "S등급 (감성 및 미각 자극)")


📊 S등급 (감성 및 미각 자극) A/B 테스트 결과


[1/60] S-1
--------------------------------------------------------------------------------
한글 입력: "숯불에 구운 삼겹살, 육즙이 뚝뚝 떨어지는 모습"

대조군 A: "A mouth-watering scene featuring perfectly grilled pork belly, its golden-brown, crispy edges glistening with savory juices. The pork belly slices are arranged on a traditional Korean grill, with charred marks enhancing their appetizing appearance. Juices are visibly dripping off the sizzling meat, creating a tantalizing effect. The setting is an outdoor barbecue area, surrounded by lush greenery, evoking a warm and inviting atmosphere. Soft, natural lighting filters through the trees, casting g..."

실험군 B: "Charcoal-grilled samgyeopsal pork belly sizzling on an open charcoal grate, a single thick slice held at a 45-degree macro close-up as golden juices bead on the caramelized surface and drip mid-air into glowing embers below. Shot on a Canon EOS R5 with a 100mm f/2.8 macro lens, fast shutter to freeze falling drople

### 3.2 A등급 테스트 (비주얼 변화 및 자기관리)

In [8]:
# A등급 A/B 테스트 실행
a_grade_results = run_ab_test(A_GRADE_TEST_CASES, "A등급 (비주얼 변화 및 자기관리)")


📊 A등급 (비주얼 변화 및 자기관리) A/B 테스트 결과


[1/40] A-1
--------------------------------------------------------------------------------
한글 입력: "미용실 봄 헤어 스타일, 웨이브 펌 시술 결과"

대조군 A: "A vibrant and stylish spring hair salon setting showcasing a beautiful model with flowing, wavy perm hairstyles. The salon is illuminated by soft, natural light streaming in through large windows adorned with pastel-colored curtains, creating a warm and inviting atmosphere. The model is seated comfortably in a sleek salon chair, with a look of satisfaction as she admires her new hairstyle in a large round mirror. The waves in her hair are glossy and voluminous, cascading elegantly down her shoul..."

실험군 B: "A close-up portrait of a young woman in a high-end salon showing the result of a spring wave perm, her glossy loose waves cascading over one shoulder as she turns slightly toward the camera with a calm, confident expression. Shot like a professional beauty editorial on a full-frame camera with an 85mm short-telep

### 3.3 B+C등급 테스트 (공간 체험/교육 + 신뢰 기반 서비스)

In [9]:
# B+C등급 A/B 테스트 실행
bc_grade_results = run_ab_test(BC_GRADE_TEST_CASES, "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)")


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) A/B 테스트 결과


[1/60] B-1
--------------------------------------------------------------------------------
한글 입력: "입시학원 홍보, 소수정예 수학 클래스"

대조군 A: "A modern, well-lit classroom filled with students engaged in a small, focused mathematics class. The room has large windows allowing natural light to pour in, creating a bright and inviting atmosphere. Each student, a diverse group of young learners, is seated at individual desks, intently working on math problems with notebooks and calculators in front of them. The teacher, a middle-aged woman with glasses, is standing at the front, enthusiastically explaining a concept on a whiteboard filled w..."

실험군 B: "Entrance exam academy promotional interior showing a small elite math class where a teacher leans over a clustered table guiding five focused students, each working intently with notebooks and tablets. Shot like a professional architectural interior photograph on a Canon EOS R5 with a 24mm wide-angle lens to 

### 3.4 D+E등급 테스트 (목적형 소매 + 인프라)

In [10]:
# D+E등급 A/B 테스트 실행
de_grade_results = run_ab_test(DE_GRADE_TEST_CASES, "D+E등급 (목적형 소매 + 인프라)")


📊 D+E등급 (목적형 소매 + 인프라) A/B 테스트 결과


[1/50] D-1
--------------------------------------------------------------------------------
한글 입력: "타이어 전문점, 사계절 타이어 할인"

대조군 A: "A vibrant tire shop bustling with activity, showcasing a wide array of all-season tires prominently displayed. The scene is set in a well-lit, spacious garage with polished concrete floors and colorful walls adorned with tire-themed artwork. Sunlight streams through large windows, casting playful shadows on the tires stacked high in neat rows. Customers, a diverse group of people, are interacting with friendly staff members, examining the tires while discussing their features. In the background,..."

실험군 B: "All-season tires arranged prominently in a professional tire specialty shop, a single tire angled toward the camera to showcase deep tread patterns and sidewall detail while several others sit neatly stacked on metal racks and wooden pallets in the background. Shot on a Canon EOS R5 with a 50mm f/1.8 lens, low-angle c

## 4. 종합 분석 및 결과 저장

In [11]:
import pandas as pd
from datetime import datetime

# 모든 결과 합치기
all_results = s_grade_results + a_grade_results + bc_grade_results + de_grade_results

# DataFrame으로 변환
df = pd.DataFrame(all_results)

# 기본 통계
print("\n" + "="*100)
print("📊 전체 A/B 테스트 종합 결과")
print("="*100)

print(f"\n총 테스트 케이스: {len(all_results)}개")
print(f"\n대조군 A 평균 길이: {df['len_a'].mean():.1f} 단어")
print(f"실험군 B 평균 길이: {df['len_b'].mean():.1f} 단어")
print(f"\n대조군 A 길이 범위: {df['len_a'].min()} ~ {df['len_a'].max()} 단어")
print(f"실험군 B 길이 범위: {df['len_b'].min()} ~ {df['len_b'].max()} 단어")

# 그룹별 통계
print("\n" + "-"*50)
print("그룹별 평균 길이 비교")
print("-"*50)
group_stats = df.groupby('group').agg({
    'len_a': 'mean',
    'len_b': 'mean'
}).round(1)
group_stats.columns = ['대조군 A 평균', '실험군 B 평균']
print(group_stats)

# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"{timestamp}_ab_test_image_prompt_results.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 결과 저장 완료: {output_path}")


📊 전체 A/B 테스트 종합 결과

총 테스트 케이스: 210개

대조군 A 평균 길이: 131.0 단어
실험군 B 평균 길이: 176.1 단어

대조군 A 길이 범위: 106 ~ 158 단어
실험군 B 길이 범위: 150 ~ 220 단어

--------------------------------------------------
그룹별 평균 길이 비교
--------------------------------------------------
       대조군 A 평균  실험군 B 평균
group                    
A-1       131.6     179.0
A-2       131.1     179.9
A-3       133.0     179.1
A-4       129.6     176.8
B-1       128.9     176.4
B-2       133.9     181.7
B-3       138.1     175.2
C-1       129.2     176.6
C-2       134.2     180.5
C-3       126.4     182.2
D-1       127.8     173.3
D-2       128.1     175.0
D-3       133.0     170.9
E-1       126.5     175.2
E-2       123.1     164.4
S-1       126.8     176.4
S-2       132.6     172.2
S-3       128.1     172.0
S-4       136.2     179.8
S-5       133.7     180.1
S-6       139.1     172.2

💾 결과 저장 완료: 20260126_231811_ab_test_image_prompt_results.csv


## 5. GPT 기반 프롬프트 품질 평가

### 5.1 평가 기준 정의 (업종별 분류 문서 기반)

In [12]:
# ============================================
# GPT 기반 프롬프트 품질 평가 시스템
# ============================================

# 등급별 평가 기준 (업종별_분류_및_광고전략.md 기반)
EVALUATION_CRITERIA = {
    "S": {
        "name": "S등급 - 감성 및 미각 자극",
        "criteria": """
## S등급 평가 기준 (감성 및 미각 자극)

### 핵심 원칙
- 이성적인 판단보다 '시각적 자극'에 의한 즉각적 반응 유도
- 사진/영상 한 장의 퀄리티가 광고 성과의 80% 이상 결정

### 세부 평가 항목 (각 20점, 총 100점)

1. **질감 표현 (Texture)** - 20점
   - 음식의 육즙, 윤기, 바삭함 등 질감이 구체적으로 묘사되었는가
   - S-1: 육즙(juicy), 지글거림(sizzling), 숯불 자국(grill marks)
   - S-2: 신선함(fresh), 투명한 색감(translucent), 물방울(water droplets)
   - S-3: 부드러운 질감(creamy), 촉촉함(moist), 파스텔 톤
   - S-4: 정갈함(neat), 다채로움(colorful), 균형잡힌 배치

2. **조명 설정 (Lighting)** - 20점
   - 업종에 맞는 조명 설정이 명시되었는가
   - 따뜻한 조명, 자연광, 스튜디오 조명 등 구체적 명시

3. **색감/분위기 (Color/Mood)** - 20점
   - 식욕을 자극하는 색감 또는 감성적 분위기가 표현되었는가
   - 따뜻한 톤, 선명한 색상, 부드러운 파스텔 등

4. **구도/앵글 (Composition)** - 20점
   - 클로즈업, 45도 앵글, 탑다운 뷰 등 구도가 명시되었는가
   - 음식 중심 배치, 보케 효과 등

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어나 키워드가 포함되었는가
"""
    },
    "A": {
        "name": "A등급 - 비주얼 변화 및 자기관리",
        "criteria": """
## A등급 평가 기준 (비주얼 변화 및 자기관리)

### 핵심 원칙
- '변화'와 '결과'를 시각적으로 증명
- 피부결, 머릿결, 근육 선명도 등 디테일 강조

### 세부 평가 항목 (각 20점, 총 100점)

1. **변화/결과 표현 (Transformation)** - 20점
   - A-1: 피부결 윤기, 머릿결 광택, 아름다운 네일 디자인
   - A-2: 근육 선명도, 역동적 자세, 건강한 체형
   - A-3: 원단 질감, 스타일링, 트렌디한 코디
   - A-4: 꽃의 생생함, 반려동물 털 질감, 섬세한 디테일

2. **조명 설정 (Lighting)** - 20점
   - A-1: 스튜디오 림 라이팅, 얕은 피사계 심도
   - A-2: 시네마틱 헬스장 조명, 높은 대비
   - A-3: 자연광, 패션 룩북 스타일
   - A-4: 부드러운 자연광, 매크로 촬영

3. **인물/제품 중심성 (Subject Focus)** - 20점
   - 주요 피사체가 명확히 부각되었는가
   - 배경과의 적절한 분리

4. **전문성/신뢰감 (Professionalism)** - 20점
   - 전문적인 느낌이 전달되는가
   - 고급스러운 분위기

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어나 키워드가 포함되었는가
"""
    },
    "BC": {
        "name": "B+C등급 - 공간 체험/신뢰 기반 서비스",
        "criteria": """
## B+C등급 평가 기준 (공간 체험 + 신뢰 기반 서비스)

### B등급 핵심 원칙
- 단일 제품이 아닌 '머무는 경험'과 '성취'를 판매
- 광각 렌즈를 활용한 공간감 표현

### C등급 핵심 원칙
- 화려함보다는 '실패 없는 선택'임을 증명하는 신뢰성
- 전문적인 블루톤 배색, 정돈된 레이아웃

### 세부 평가 항목 (각 20점, 총 100점)

1. **공간감/시설 표현 (Space/Facility)** - 20점
   - B-1: 밝고 쾌적한 학습 공간, 정돈된 책상
   - B-2: 시네마틱 인테리어, 럭셔리한 숙박 시설
   - B-3: 넓은 시설, 전문 장비, 현대적 느낌
   - C등급: 미니멀 기업 스타일, 청결한 의료 공간

2. **조명 설정 (Lighting)** - 20점
   - B등급: 광각, 시네마틱 조명, 무드 조명
   - C등급: 밝고 균일한 조명, 전문적 분위기

3. **신뢰감/전문성 (Trust/Expertise)** - 20점
   - 전문적이고 믿을 수 있는 느낌이 전달되는가
   - C등급: 블루 계열 액센트, 깨끗한 환경

4. **구도/앵글 (Composition)** - 20점
   - 광각 렌즈, 파노라마 뷰, 시설 전경 등

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어나 키워드가 포함되었는가
"""
    },
    "DE": {
        "name": "D+E등급 - 목적형 소매/인프라",
        "criteria": """
## D+E등급 평가 기준 (목적형 소매 + 인프라)

### D등급 핵심 원칙
- 견고함, 실용성, 기능성 강조
- 산업적 미학, 높은 대비, 금속 질감

### E등급 핵심 원칙
- 접근성, 편의성, 품위
- 직관적 사진, 명확한 간판, 차분한 분위기

### 세부 평가 항목 (각 20점, 총 100점)

1. **제품/시설 명확성 (Clarity)** - 20점
   - D-1: 금속 질감, 내구성, 산업적 미학
   - D-2: 깨끗한 제품 촬영, 첨단 느낌
   - E-1: 직관적 사진, 명확한 간판
   - E-2: 차분한 분위기, 절제된 색상, 대칭 구도

2. **조명 설정 (Lighting)** - 20점
   - D등급: 밝은 조명, 스튜디오 조명, 높은 대비
   - E등급: 주간 자연광, 차분한 조명

3. **실용성/기능성 (Functionality)** - 20점
   - 제품의 실용성과 기능이 잘 전달되는가

4. **정돈된 구성 (Organization)** - 20점
   - 정리된 선반, 깔끔한 배치, 명확한 구성

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어나 키워드가 포함되었는가
"""
    }
}

print("평가 기준 로드 완료!")

평가 기준 로드 완료!


### 5.2 GPT 평가 함수 정의

In [13]:
def evaluate_prompts_with_gpt(
    korean_input: str,
    prompt_a: str,
    prompt_b: str,
    group: str
) -> Dict:
    """
    GPT를 사용하여 두 프롬프트를 비교 평가

    Args:
        korean_input: 원본 한글 입력
        prompt_a: 대조군 A 프롬프트
        prompt_b: 실험군 B 프롬프트
        group: 그룹 코드 (S-1, A-2 등)

    Returns:
        Dict: 평가 결과
    """
    # 등급 결정
    if group.startswith("S"):
        grade = "S"
    elif group.startswith("A"):
        grade = "A"
    elif group.startswith("B") or group.startswith("C"):
        grade = "BC"
    else:
        grade = "DE"

    criteria = EVALUATION_CRITERIA[grade]["criteria"]
    grade_name = EVALUATION_CRITERIA[grade]["name"]

    system_prompt = f"""당신은 광고용 이미지 생성 프롬프트 품질을 평가하는 전문가입니다.
소상공인 광고 이미지 생성을 위한 프롬프트의 적합성을 평가합니다.

{criteria}

## 평가 방법
1. 각 항목별로 0-20점 척도로 평가
2. 두 프롬프트(A와 B)를 동일한 기준으로 평가
3. 어떤 프롬프트가 해당 업종의 광고 이미지 생성에 더 적합한지 판단
4. 반드시 JSON 형식으로 출력

## 출력 형식 (JSON)
{{
    "grade": "{grade_name}",
    "evaluation_a": {{
        "texture_or_subject": <0-20>,
        "lighting": <0-20>,
        "color_mood": <0-20>,
        "composition": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "evaluation_b": {{
        "texture_or_subject": <0-20>,
        "lighting": <0-20>,
        "color_mood": <0-20>,
        "composition": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "winner": "A" 또는 "B" 또는 "TIE",
    "winner_reason": "선택 이유 (2-3문장)",
    "detailed_comparison": "두 프롬프트의 상세 비교 분석 (3-4문장)"
}}
"""

    user_prompt = f"""## 평가 대상

### 원본 한글 입력 (광고 요청)
"{korean_input}"

### 업종 그룹
{group}

### 대조군 A 프롬프트
"{prompt_a}"

### 실험군 B 프롬프트
"{prompt_b}"

위 두 프롬프트를 평가 기준에 따라 비교 분석하고, 어떤 프롬프트가 해당 업종의 광고 이미지 생성에 더 적합한지 판단해주세요.
반드시 JSON 형식으로만 응답해주세요."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # 더 정확한 평가를 위해 gpt-4o 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # 일관된 평가를 위해 낮은 temperature
            response_format={"type": "json_object"}
        )

        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        return {"error": str(e)}


def evaluate_existing_results(existing_results: List[Dict], group_name: str, sample_size: int = None):
    """
    이미 생성된 A/B 테스트 결과를 사용하여 GPT 평가만 수행
    
    Args:
        existing_results: 섹션 3에서 생성된 A/B 테스트 결과 리스트
        group_name: 그룹명
        sample_size: 평가할 샘플 수 (None이면 전체 평가)
    
    Returns:
        평가 결과 리스트, 승패 통계
    """
    import random
    import time
    
    print(f"\n{'='*100}")
    print(f"📊 {group_name} GPT 평가 (기존 A/B 결과 활용)")
    print(f"{'='*100}\n")
    
    # 샘플 선택 (None이면 전체)
    if sample_size and sample_size < len(existing_results):
        samples = random.sample(existing_results, sample_size)
        print(f"📌 전체 {len(existing_results)}개 중 {sample_size}개 샘플 평가")
    else:
        samples = existing_results
        print(f"📌 전체 {len(existing_results)}개 평가")
    
    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}
    
    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("korean_input", "")
        prompt_a = case.get("prompt_a", "")
        prompt_b = case.get("prompt_b", "")
        
        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')
        print(f'\n대조군 A: "{prompt_a[:200]}..."' if len(prompt_a) > 200 else f'\n대조군 A: "{prompt_a}"')
        print(f'\n실험군 B: "{prompt_b[:200]}..."' if len(prompt_b) > 200 else f'\n실험군 B: "{prompt_b}"')
        
        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_prompts_with_gpt(korean_input, prompt_a, prompt_b, group)
        
        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")
            
            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
            print(f"   📝 이유: {evaluation.get('winner_reason', 'N/A')}")
            
            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))
            
            results.append({
                "group": group,
                "korean_input": korean_input,
                "prompt_a": prompt_a,
                "prompt_b": prompt_b,
                "len_a": case.get("len_a", len(prompt_a.split())),
                "len_b": case.get("len_b", len(prompt_b.split())),
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")
            # 오류가 나도 결과에 포함 (평가 없이)
            results.append({
                "group": group,
                "korean_input": korean_input,
                "prompt_a": prompt_a,
                "prompt_b": prompt_b,
                "len_a": case.get("len_a", len(prompt_a.split())),
                "len_b": case.get("len_b", len(prompt_b.split())),
                "score_a": None,
                "score_b": None,
                "winner": None,
                "evaluation": evaluation
            })
        
        # API 레이트 리밋 방지를 위한 딜레이
        if i < len(samples):
            time.sleep(0.5)
    
    # 요약 통계
    valid_results = [r for r in results if r.get("winner")]
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 평가 요약")
    print(f"{'='*100}")
    print(f"평가 케이스 수: {len(valid_results)}/{len(samples)}")
    
    if valid_results:
        print(f"\n🏆 승패 현황:")
        print(f"   대조군 A 승: {wins['A']}회 ({wins['A']/len(valid_results)*100:.1f}%)")
        print(f"   실험군 B 승: {wins['B']}회 ({wins['B']/len(valid_results)*100:.1f}%)")
        print(f"   무승부: {wins['TIE']}회 ({wins['TIE']/len(valid_results)*100:.1f}%)")
    
    if total_scores["A"] and total_scores["B"]:
        print(f"\n📊 평균 점수:")
        print(f"   대조군 A: {sum(total_scores['A'])/len(total_scores['A']):.1f}점")
        print(f"   실험군 B: {sum(total_scores['B'])/len(total_scores['B']):.1f}점")
    
    return results, wins


# 기존 run_ab_test_with_evaluation 함수는 유지 (필요시 사용)
def run_ab_test_with_evaluation(test_cases: List[Dict], group_name: str, sample_size: int = 5):
    """
    A/B 테스트 실행 + GPT 평가 (새로 생성하면서 평가)
    - 주의: 이 함수는 A/B 결과를 새로 생성합니다.
    - 기존 결과를 활용하려면 evaluate_existing_results()를 사용하세요.
    """
    import random

    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 + GPT 평가 (새로 생성)")
    print(f"{'='*100}\n")

    samples = random.sample(test_cases, min(sample_size, len(test_cases)))

    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}

    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")

        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')

        # A/B 프롬프트 생성
        prompt_a = generate_prompt_control_a(korean_input)
        prompt_b = generate_prompt_experiment_b(korean_input)

        print(f'\n대조군 A: "{prompt_a[:200]}..."' if len(prompt_a) > 200 else f'\n대조군 A: "{prompt_a}"')
        print(f'\n실험군 B: "{prompt_b[:200]}..."' if len(prompt_b) > 200 else f'\n실험군 B: "{prompt_b}"')

        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_prompts_with_gpt(korean_input, prompt_a, prompt_b, group)

        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")

            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")

            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))

            results.append({
                "group": group,
                "korean_input": korean_input,
                "prompt_a": prompt_a,
                "prompt_b": prompt_b,
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")

    return results, wins

### 5.3 S등급 GPT 평가 실행

In [14]:
# S등급 GPT 평가 - 섹션 3에서 생성된 s_grade_results 활용
# sample_size=None 으로 설정하면 전체 40개 평가 (API 비용 주의)
# sample_size=10 으로 설정하면 10개만 샘플링하여 평가

s_eval_results, s_wins = evaluate_existing_results(
    s_grade_results,  # 섹션 3에서 생성된 결과 사용
    "S등급 (감성 및 미각 자극)",
    sample_size=None  # 전체 평가 (또는 원하는 샘플 수로 변경)
)


📊 S등급 (감성 및 미각 자극) GPT 평가 (기존 A/B 결과 활용)

📌 전체 60개 평가

[1/60] S-1
--------------------------------------------------------------------------------
한글 입력: "숯불에 구운 삼겹살, 육즙이 뚝뚝 떨어지는 모습"

대조군 A: "A mouth-watering scene featuring perfectly grilled pork belly, its golden-brown, crispy edges glistening with savory juices. The pork belly slices are arranged on a traditional Korean grill, with char..."

실험군 B: "Charcoal-grilled samgyeopsal pork belly sizzling on an open charcoal grate, a single thick slice held at a 45-degree macro close-up as golden juices bead on the caramelized surface and drip mid-air in..."

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 87점
   실험군 B 점수: 98점
   🏆 승자: 실험군 B
   📝 이유: 프롬프트 B는 질감, 조명, 색감, 구도 등 모든 면에서 매우 구체적이고 전문적인 표현을 제공하여 고급스러운 광고 이미지를 생성하는 데 더 적합하다.

[2/60] S-1
--------------------------------------------------------------------------------
한글 입력: "치킨 전문점 홍보, 갓 튀긴 바삭한 치킨"

대조군 A: "A vibrant, inviting scene inside a cozy chicken specialty restaurant. The focus is on a

### 5.4 A등급 GPT 평가 실행

In [15]:
# A등급 GPT 평가 - 섹션 3에서 생성된 a_grade_results 활용
a_eval_results, a_wins = evaluate_existing_results(
    a_grade_results,  # 섹션 3에서 생성된 결과 사용
    "A등급 (비주얼 변화 및 자기관리)",
    sample_size=None  # 전체 평가
)


📊 A등급 (비주얼 변화 및 자기관리) GPT 평가 (기존 A/B 결과 활용)

📌 전체 40개 평가

[1/40] A-1
--------------------------------------------------------------------------------
한글 입력: "미용실 봄 헤어 스타일, 웨이브 펌 시술 결과"

대조군 A: "A vibrant and stylish spring hair salon setting showcasing a beautiful model with flowing, wavy perm hairstyles. The salon is illuminated by soft, natural light streaming in through large windows ador..."

실험군 B: "A close-up portrait of a young woman in a high-end salon showing the result of a spring wave perm, her glossy loose waves cascading over one shoulder as she turns slightly toward the camera with a cal..."

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 86점
   실험군 B 점수: 94점
   🏆 승자: 실험군 B
   📝 이유: 프롬프트 B는 텍스처와 조명에 대한 세부적인 설명을 통해 더 전문적이고 신뢰감을 주는 이미지를 생성할 수 있습니다.

[2/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부관리실 홍보, 맑고 깨끗한 피부 관리"

대조군 A: "A serene skincare clinic interior, featuring a bright and airy atmosphere with large windows allowing na

### 5.5 B+C등급 GPT 평가 실행

In [16]:
# B+C등급 GPT 평가 - 섹션 3에서 생성된 bc_grade_results 활용
bc_eval_results, bc_wins = evaluate_existing_results(
    bc_grade_results,  # 섹션 3에서 생성된 결과 사용
    "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)",
    sample_size=None  # 전체 평가
)


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) GPT 평가 (기존 A/B 결과 활용)

📌 전체 60개 평가

[1/60] B-1
--------------------------------------------------------------------------------
한글 입력: "입시학원 홍보, 소수정예 수학 클래스"

대조군 A: "A modern, well-lit classroom filled with students engaged in a small, focused mathematics class. The room has large windows allowing natural light to pour in, creating a bright and inviting atmosphere..."

실험군 B: "Entrance exam academy promotional interior showing a small elite math class where a teacher leans over a clustered table guiding five focused students, each working intently with notebooks and tablets..."

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 84점
   실험군 B 점수: 92점
   🏆 승자: 실험군 B
   📝 이유: 프롬프트 B는 광각 렌즈와 조명 설정을 통해 공간감을 잘 표현하고, 전문적인 느낌을 강조하여 입시학원의 소수정예 수업의 장점을 잘 드러냅니다.

[2/60] B-1
--------------------------------------------------------------------------------
한글 입력: "외국어 학원, 원어민 영어 회화 수업"

대조군 A: "A vibrant and inviting classroom setting designed for foreign language learning, featur

### 5.6 D+E등급 GPT 평가 실행

In [17]:
# D+E등급 GPT 평가 - 섹션 3에서 생성된 de_grade_results 활용
de_eval_results, de_wins = evaluate_existing_results(
    de_grade_results,  # 섹션 3에서 생성된 결과 사용
    "D+E등급 (목적형 소매 + 인프라)",
    sample_size=None  # 전체 평가
)


📊 D+E등급 (목적형 소매 + 인프라) GPT 평가 (기존 A/B 결과 활용)

📌 전체 50개 평가

[1/50] D-1
--------------------------------------------------------------------------------
한글 입력: "타이어 전문점, 사계절 타이어 할인"

대조군 A: "A vibrant tire shop bustling with activity, showcasing a wide array of all-season tires prominently displayed. The scene is set in a well-lit, spacious garage with polished concrete floors and colorfu..."

실험군 B: "All-season tires arranged prominently in a professional tire specialty shop, a single tire angled toward the camera to showcase deep tread patterns and sidewall detail while several others sit neatly ..."

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 81점
   실험군 B 점수: 92점
   🏆 승자: 실험군 B
   📝 이유: 프롬프트 B는 타이어의 질감과 세부 사항을 강조하여 제품의 내구성과 기능성을 잘 전달하며, 조명과 색상도 산업적 미학을 잘 반영하고 있다.

[2/50] D-1
--------------------------------------------------------------------------------
한글 입력: "자동차 부품점, 순정 브레이크 패드"

대조군 A: "An automotive parts store interior showcasing a well-organized display of genuine brake pads. The s

### 5.7 전체 GPT 평가 종합 결과

In [18]:
# 전체 결과 종합
all_eval_results = s_eval_results + a_eval_results + bc_eval_results + de_eval_results
all_wins = {
    "A": s_wins["A"] + a_wins["A"] + bc_wins["A"] + de_wins["A"],
    "B": s_wins["B"] + a_wins["B"] + bc_wins["B"] + de_wins["B"],
    "TIE": s_wins["TIE"] + a_wins["TIE"] + bc_wins["TIE"] + de_wins["TIE"]
}

total_evaluated = len(all_eval_results)

print("\n" + "="*100)
print("🏆 전체 GPT 평가 종합 결과")
print("="*100)

print(f"\n총 평가 케이스: {total_evaluated}개")
print(f"\n📊 전체 승패 현황:")
print(f"   대조군 A 승: {all_wins['A']}회 ({all_wins['A']/total_evaluated*100:.1f}%)")
print(f"   실험군 B 승: {all_wins['B']}회 ({all_wins['B']/total_evaluated*100:.1f}%)")
print(f"   무승부: {all_wins['TIE']}회 ({all_wins['TIE']/total_evaluated*100:.1f}%)")

# 등급별 결과 테이블
print("\n" + "-"*50)
print("등급별 승패 현황")
print("-"*50)
grade_results = pd.DataFrame({
    "등급": ["S등급", "A등급", "B+C등급", "D+E등급", "전체"],
    "대조군 A 승": [s_wins["A"], a_wins["A"], bc_wins["A"], de_wins["A"], all_wins["A"]],
    "실험군 B 승": [s_wins["B"], a_wins["B"], bc_wins["B"], de_wins["B"], all_wins["B"]],
    "무승부": [s_wins["TIE"], a_wins["TIE"], bc_wins["TIE"], de_wins["TIE"], all_wins["TIE"]]
})
print(grade_results.to_string(index=False))

# 평균 점수 계산
all_scores_a = [r["score_a"] for r in all_eval_results if r.get("score_a")]
all_scores_b = [r["score_b"] for r in all_eval_results if r.get("score_b")]

if all_scores_a and all_scores_b:
    print(f"\n📈 전체 평균 점수:")
    print(f"   대조군 A: {sum(all_scores_a)/len(all_scores_a):.1f}점")
    print(f"   실험군 B: {sum(all_scores_b)/len(all_scores_b):.1f}점")
    print(f"   점수 차이: {sum(all_scores_b)/len(all_scores_b) - sum(all_scores_a)/len(all_scores_a):+.1f}점")

# 결과 저장
eval_df = pd.DataFrame([{
    "group": r["group"],
    "korean_input": r["korean_input"],
    "score_a": r["score_a"],
    "score_b": r["score_b"],
    "winner": r["winner"],
    "prompt_a": r["prompt_a"][:500],
    "prompt_b": r["prompt_b"][:500]
} for r in all_eval_results])

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
eval_output_path = f"{timestamp}_ab_test_image_prompt_gpt_evaluation.csv"
eval_df.to_csv(eval_output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 GPT 평가 결과 저장 완료: {eval_output_path}")


🏆 전체 GPT 평가 종합 결과

총 평가 케이스: 210개

📊 전체 승패 현황:
   대조군 A 승: 2회 (1.0%)
   실험군 B 승: 208회 (99.0%)
   무승부: 0회 (0.0%)

--------------------------------------------------
등급별 승패 현황
--------------------------------------------------
   등급  대조군 A 승  실험군 B 승  무승부
  S등급        0       60    0
  A등급        1       39    0
B+C등급        1       59    0
D+E등급        0       50    0
   전체        2      208    0

📈 전체 평균 점수:
   대조군 A: 84.5점
   실험군 B: 94.0점
   점수 차이: +9.6점

💾 GPT 평가 결과 저장 완료: 20260126_235253_ab_test_image_prompt_gpt_evaluation.csv


## 7. 결론

### 실험 설계

| 항목 | 대조군 A | 실험군 B |
|------|---------|----------|
| 방식 | 일반 OpenAI API | 업종별 특화 시스템 |
| 프롬프트 구조 | 단순 번역 + 일반 지시문 | 18개 하위 그룹별 최적화 |
| 업종 키워드 | X | O (YAML 기반) |
| 조명/색감 가이드 | X | O (업종별 특화) |
| 구도/스타일 | 일반적 | 업종 맞춤형 |

### GPT 평가 기준 (업종별_분류_및_광고전략.md 기반)

| 평가 항목 | 배점 | 설명 |
|----------|------|------|
| 질감/주제 표현 | 20점 | 업종별 핵심 비주얼 요소 표현 |
| 조명 설정 | 20점 | 업종에 맞는 조명 명시 |
| 색감/분위기 | 20점 | 적절한 색감과 무드 표현 |
| 구도/앵글 | 20점 | 촬영 구도와 앵글 명시 |
| 업종 특화 키워드 | 20점 | 업종별 전문 용어 포함 |
| **총점** | **100점** | |

### 기대 효과

1. **업종별 최적화**: 실험군 B는 각 업종의 특성을 반영한 키워드, 조명, 구도를 자동 적용
2. **일관된 품질**: 동일 업종 내에서 일관된 비주얼 스타일 유지
3. **전문성**: 업종별 전문 용어와 비주얼 문법 활용
4. **객관적 검증**: GPT 기반 자동 평가로 품질 차이를 정량적으로 측정

## 8. B 실험군 패배/무승부 케이스 분석

실험군 B(업종별 특화 시스템)가 대조군 A(일반 OpenAI API)에게 패배하거나 무승부인 케이스를 분석합니다.
이 케이스들은 `industries.yaml` 템플릿 개선의 핵심 데이터가 됩니다.

In [19]:
# ============================================
# B 실험군 패배/무승부 케이스 수집 및 분석
# ============================================

def collect_b_loss_or_tie_cases(all_eval_results: List[Dict]) -> List[Dict]:
    """
    B 실험군이 패배하거나 무승부인 케이스 수집
    
    Args:
        all_eval_results: 전체 GPT 평가 결과 리스트
    
    Returns:
        B가 지거나 비긴 케이스 리스트
    """
    b_loss_or_tie = []
    
    for result in all_eval_results:
        winner = result.get("winner")
        if winner in ["A", "TIE"]:  # A가 이기거나 무승부
            b_loss_or_tie.append(result)
    
    return b_loss_or_tie


def display_b_loss_cases(cases: List[Dict], title: str = "B 실험군 패배/무승부 케이스"):
    """
    B 실험군 패배/무승부 케이스를 상세히 출력
    """
    print(f"\n{'='*120}")
    print(f"📊 {title}")
    print(f"{'='*120}")
    print(f"총 {len(cases)}개 케이스 (A 승리 또는 무승부)\n")
    
    # 승패별 분류
    a_wins = [c for c in cases if c.get("winner") == "A"]
    ties = [c for c in cases if c.get("winner") == "TIE"]
    
    print(f"  - A 승리: {len(a_wins)}개")
    print(f"  - 무승부: {len(ties)}개")
    
    # 그룹별 분류
    group_counts = {}
    for case in cases:
        group = case.get("group", "Unknown")
        group_counts[group] = group_counts.get(group, 0) + 1
    
    print(f"\n📌 그룹별 분포:")
    for group, count in sorted(group_counts.items()):
        print(f"  - {group}: {count}개")
    
    print(f"\n{'='*120}")
    print("상세 케이스 분석")
    print(f"{'='*120}\n")
    
    for i, case in enumerate(cases, 1):
        group = case.get("group", "Unknown")
        korean_input = case.get("korean_input", "N/A")
        prompt_a = case.get("prompt_a", "N/A")
        prompt_b = case.get("prompt_b", "N/A")
        score_a = case.get("score_a", "N/A")
        score_b = case.get("score_b", "N/A")
        winner = case.get("winner", "N/A")
        evaluation = case.get("evaluation", {})
        
        winner_label = "🏆 A 승리" if winner == "A" else "🤝 무승부"
        
        print(f"\n[{i}/{len(cases)}] {group} - {winner_label}")
        print(f"-" * 100)
        
        # 입력 및 결과
        print(f"\n📝 한글 입력:")
        print(f"   \"{korean_input}\"")
        
        print(f"\n🅰️ 대조군 A 프롬프트 (점수: {score_a}점):")
        print(f"   \"{prompt_a[:300]}{'...' if len(str(prompt_a)) > 300 else ''}\"")
        
        print(f"\n🅱️ 실험군 B 프롬프트 (점수: {score_b}점):")
        print(f"   \"{prompt_b[:300]}{'...' if len(str(prompt_b)) > 300 else ''}\"")
        
        # GPT 평가 상세 내용
        if evaluation and "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            
            print(f"\n📊 GPT 평가 상세:")
            print(f"   [대조군 A]")
            print(f"   - 질감/피사체: {eval_a.get('texture_or_subject', 'N/A')}점")
            print(f"   - 조명: {eval_a.get('lighting', 'N/A')}점")
            print(f"   - 색감/분위기: {eval_a.get('color_mood', 'N/A')}점")
            print(f"   - 구도: {eval_a.get('composition', 'N/A')}점")
            print(f"   - 업종 특화: {eval_a.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_a.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_a.get('weaknesses', 'N/A')}")
            
            print(f"\n   [실험군 B]")
            print(f"   - 질감/피사체: {eval_b.get('texture_or_subject', 'N/A')}점")
            print(f"   - 조명: {eval_b.get('lighting', 'N/A')}점")
            print(f"   - 색감/분위기: {eval_b.get('color_mood', 'N/A')}점")
            print(f"   - 구도: {eval_b.get('composition', 'N/A')}점")
            print(f"   - 업종 특화: {eval_b.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_b.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_b.get('weaknesses', 'N/A')}")
            
            print(f"\n   📝 승패 이유: {evaluation.get('winner_reason', 'N/A')}")
            print(f"   📝 상세 비교: {evaluation.get('detailed_comparison', 'N/A')}")
        
        print(f"\n{'─'*100}")
    
    return cases


# B 패배/무승부 케이스 수집
b_loss_or_tie_cases = collect_b_loss_or_tie_cases(all_eval_results)

# 상세 출력
display_b_loss_cases(b_loss_or_tie_cases, "이미지 프롬프트 B 실험군 패배/무승부 케이스")


📊 이미지 프롬프트 B 실험군 패배/무승부 케이스
총 2개 케이스 (A 승리 또는 무승부)

  - A 승리: 2개
  - 무승부: 0개

📌 그룹별 분포:
  - A-3: 1개
  - B-3: 1개

상세 케이스 분석


[1/2] A-3 - 🏆 A 승리
----------------------------------------------------------------------------------------------------

📝 한글 입력:
   "시각 디자인업, 브랜드 로고 디자인"

🅰️ 대조군 A 프롬프트 (점수: 88점):
   "Create an image showcasing a vibrant and modern graphic design workspace dedicated to brand logo design. The setting is a bright, airy studio filled with large windows allowing natural light to flood in, casting soft shadows on the wooden floor. On a sleek, minimalist desk, an array of colorful desi..."

🅱️ 실험군 B 프롬프트 (점수: 86점):
   "A high-end fashion brand emblem presented as an abstract logo mark applied across garments and accessories in an editorial lookbook scene. The emblem appears as a refined geometric symbol without any letters, shown subtly embossed on a leather label, stamped on a brushed metal pin, and embroidered o..."

📊 GPT 평가 상세:
   [대조군 A]
   - 질감/피사체: 18점
   - 조명

[{'group': 'A-3',
  'korean_input': '시각 디자인업, 브랜드 로고 디자인',
  'prompt_a': 'Create an image showcasing a vibrant and modern graphic design workspace dedicated to brand logo design. The setting is a bright, airy studio filled with large windows allowing natural light to flood in, casting soft shadows on the wooden floor. On a sleek, minimalist desk, an array of colorful design tools such as markers, sketch pads, and a high-end graphic tablet are neatly arranged. In the background, a large mood board is pinned to the wall, featuring various logo sketches, color palettes, and typography samples, creating an inspiring visual collage. The atmosphere is creative and dynamic, with splashes of color from paint tubes and illustrations scattered throughout the space. A designer, a young woman with an artistic flair, is focused on her work, thoughtfully sketching out ideas on paper. The overall scene conveys a sense of innovation and passion for visual design, capturing the essence of the brand log

## 9. industries.yaml 개선사항 제안

B 실험군 패배/무승부 케이스를 분석하여 `src/generation/image_generation/prompt/config/industries.yaml` 파일의 개선사항을 GPT를 활용해 구체적으로 제안합니다.

In [20]:
# ============================================
# industries.yaml 개선사항 제안 시스템
# ============================================

def generate_improvement_suggestions(
    b_loss_cases: List[Dict],
    yaml_file_path: str = "src/generation/image_generation/prompt/config/industries.yaml"
) -> Dict:
    """
    B 패배/무승부 케이스를 분석하여 industries.yaml 개선사항 제안
    
    Args:
        b_loss_cases: B가 지거나 비긴 케이스 리스트
        yaml_file_path: industries.yaml 파일 경로
    
    Returns:
        그룹별 개선사항 딕셔너리
    """
    if not b_loss_cases:
        print("분석할 B 패배/무승부 케이스가 없습니다.")
        return {}
    
    # 그룹별로 케이스 분류
    cases_by_group = {}
    for case in b_loss_cases:
        group = case.get("group", "Unknown")
        if group not in cases_by_group:
            cases_by_group[group] = []
        cases_by_group[group].append(case)
    
    print(f"\n{'='*120}")
    print(f"🔧 industries.yaml 개선사항 제안")
    print(f"{'='*120}")
    print(f"대상 파일: {yaml_file_path}")
    print(f"분석 대상: {len(b_loss_cases)}개 케이스 ({len(cases_by_group)}개 그룹)")
    print(f"{'='*120}\n")
    
    all_suggestions = {}
    
    for group, cases in cases_by_group.items():
        print(f"\n{'─'*100}")
        print(f"📌 {group} 그룹 분석 중... ({len(cases)}개 케이스)")
        print(f"{'─'*100}")
        
        # 케이스 요약 생성
        case_summaries = []
        for case in cases:
            summary = {
                "korean_input": case.get("korean_input", ""),
                "prompt_a": case.get("prompt_a", "")[:500],  # 너무 길면 자름
                "prompt_b": case.get("prompt_b", "")[:500],
                "score_a": case.get("score_a"),
                "score_b": case.get("score_b"),
                "winner": case.get("winner"),
                "evaluation": {
                    "a_strengths": case.get("evaluation", {}).get("evaluation_a", {}).get("strengths", ""),
                    "a_weaknesses": case.get("evaluation", {}).get("evaluation_a", {}).get("weaknesses", ""),
                    "b_strengths": case.get("evaluation", {}).get("evaluation_b", {}).get("strengths", ""),
                    "b_weaknesses": case.get("evaluation", {}).get("evaluation_b", {}).get("weaknesses", ""),
                    "winner_reason": case.get("evaluation", {}).get("winner_reason", ""),
                    "detailed_comparison": case.get("evaluation", {}).get("detailed_comparison", "")
                }
            }
            case_summaries.append(summary)
        
        # GPT에게 개선사항 요청
        system_prompt = """당신은 소상공인 광고 이미지 생성 시스템의 프롬프트 템플릿 전문가입니다.
A/B 테스트 결과를 분석하여 industries.yaml 파일의 개선사항을 제안합니다.

industries.yaml 파일은 다음과 같은 구조를 가집니다:
```yaml
industries:
  카테고리명:
    keywords: [키워드1, 키워드2, ...]
    lighting: "조명 설정"
    composition: "구도 설정"
    color_palette: [색상1, 색상2, ...]
    mood: "분위기"
    style: "스타일"
    prompt_template: "프롬프트 템플릿"
```

분석 결과를 바탕으로 다음을 제안해주세요:
1. 어떤 키워드를 추가/수정해야 하는지
2. 조명 설정을 어떻게 개선해야 하는지
3. 구도 설정을 어떻게 개선해야 하는지
4. 색상 팔레트를 어떻게 개선해야 하는지
5. 분위기와 스타일을 어떻게 개선해야 하는지
6. 프롬프트 템플릿을 어떻게 개선해야 하는지

반드시 JSON 형식으로 출력하세요."""

        user_prompt = f"""## 분석 대상 그룹
{group}

## A/B 테스트 결과 (B 실험군이 패배하거나 무승부인 케이스)
{json.dumps(case_summaries, ensure_ascii=False, indent=2)}

## 요청사항
위 케이스들을 분석하여 실험군 B(업종별 특화 시스템)가 대조군 A(일반 시스템)보다 성능이 떨어진 원인을 파악하고,
industries.yaml 파일의 해당 그룹 설정을 어떻게 개선해야 하는지 구체적으로 제안해주세요.

특히:
1. 대조군 A가 더 좋았던 이유 분석
2. 실험군 B의 약점 분석
3. industries.yaml에서 개선해야 할 구체적인 항목과 값

출력 형식:
{{
    "group": "{group}",
    "analysis": {{
        "why_a_won": "A가 더 좋았던 이유",
        "b_weaknesses": "B의 약점 분석",
        "common_patterns": "공통적으로 발견된 패턴"
    }},
    "improvements": {{
        "keywords": {{
            "add": ["추가할 키워드들"],
            "remove": ["제거할 키워드들"],
            "reason": "키워드 변경 이유"
        }},
        "lighting": {{
            "current_issue": "현재 조명 설정의 문제점",
            "suggestion": "개선된 조명 설정",
            "reason": "변경 이유"
        }},
        "composition": {{
            "current_issue": "현재 구도 설정의 문제점",
            "suggestion": "개선된 구도 설정",
            "reason": "변경 이유"
        }},
        "color_palette": {{
            "add": ["추가할 색상들"],
            "remove": ["제거할 색상들"],
            "reason": "색상 변경 이유"
        }},
        "mood_style": {{
            "current_issue": "현재 분위기/스타일 문제점",
            "suggestion": "개선된 분위기/스타일",
            "reason": "변경 이유"
        }},
        "prompt_template": {{
            "current_issue": "현재 템플릿 문제점",
            "suggestion": "개선된 템플릿 예시",
            "reason": "변경 이유"
        }}
    }},
    "priority": "높음/중간/낮음",
    "summary": "전체 개선사항 요약 (2-3문장)"
}}"""

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
                response_format={"type": "json_object"}
            )
            
            suggestion = json.loads(response.choices[0].message.content)
            all_suggestions[group] = suggestion
            
            # 결과 출력
            print(f"\n✅ {group} 분석 완료")
            print(f"\n📊 분석 결과:")
            print(f"   - A 승리 이유: {suggestion.get('analysis', {}).get('why_a_won', 'N/A')}")
            print(f"   - B 약점: {suggestion.get('analysis', {}).get('b_weaknesses', 'N/A')}")
            print(f"\n🔧 주요 개선사항:")
            
            improvements = suggestion.get('improvements', {})
            
            if improvements.get('keywords', {}).get('add'):
                print(f"   - 키워드 추가: {improvements['keywords']['add']}")
            if improvements.get('lighting', {}).get('suggestion'):
                print(f"   - 조명 개선: {improvements['lighting']['suggestion']}")
            if improvements.get('composition', {}).get('suggestion'):
                print(f"   - 구도 개선: {improvements['composition']['suggestion']}")
            if improvements.get('prompt_template', {}).get('suggestion'):
                print(f"   - 템플릿 개선: {improvements['prompt_template']['suggestion'][:200]}...")
            
            print(f"\n   📌 우선순위: {suggestion.get('priority', 'N/A')}")
            print(f"   📝 요약: {suggestion.get('summary', 'N/A')}")
            
        except Exception as e:
            print(f"   ❌ 분석 오류: {str(e)}")
            all_suggestions[group] = {"error": str(e)}
        
        # API 레이트 리밋 방지
        import time
        time.sleep(1)
    
    return all_suggestions


# 개선사항 생성
improvement_suggestions = generate_improvement_suggestions(b_loss_or_tie_cases)


🔧 industries.yaml 개선사항 제안
대상 파일: src/generation/image_generation/prompt/config/industries.yaml
분석 대상: 2개 케이스 (2개 그룹)


────────────────────────────────────────────────────────────────────────────────────────────────────
📌 A-3 그룹 분석 중... (1개 케이스)
────────────────────────────────────────────────────────────────────────────────────────────────────

✅ A-3 분석 완료

📊 분석 결과:
   - A 승리 이유: 프롬프트 A는 시각 디자인업에 특화된 작업 환경과 창의적 과정을 잘 표현하여 업종에 적합했습니다. 다양한 디자인 도구와 무드 보드가 브랜드 로고 디자인의 본질을 잘 나타냈습니다.
   - B 약점: 프롬프트 B는 고급 패션 브랜드에 더 적합한 키워드를 사용하여 시각 디자인업과의 직접적인 관련성이 부족했습니다. 또한, 추상적인 로고 마크와 고급스러운 질감에 집중하여 시각 디자인업의 창의적 요소를 놓쳤습니다.

🔧 주요 개선사항:
   - 키워드 추가: ['creative workspace', 'design tools', 'mood board', 'logo sketches']
   - 조명 개선: 자연광을 활용하여 밝고 통풍이 잘 되는 스튜디오 환경을 조성하고, 부드러운 그림자가 드리워지도록 설정합니다.
   - 구도 개선: 디자인 도구와 작업 공간을 중심으로 한 구도를 사용하여, 창의적 작업 과정을 강조합니다.
   - 템플릿 개선: 브랜드 로고 디자인의 창의적 과정을 강조하는 작업 환경을 포함하는 템플릿을 사용합니다....

   📌 우선순위: 높음
   📝 요약: 시각 디자인업에 맞는 창의적이고 역동적인 작업 환경을 강조하기 위해, 관련 키워드와 조명, 구도, 색상 팔레트를 조정합니

In [21]:
# ============================================
# 개선사항 종합 보고서 저장
# ============================================

def save_improvement_report(
    suggestions: Dict,
    b_loss_cases: List[Dict],
    output_path: str = "260122_이미지프롬프트_개선사항_보고서.json"
):
    """
    개선사항 보고서를 JSON 파일로 저장
    """
    report = {
        "report_type": "이미지 프롬프트 템플릿 개선사항",
        "target_file": "src/generation/image_generation/prompt/config/industries.yaml",
        "analysis_date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
        "total_b_loss_cases": len(b_loss_cases),
        "groups_analyzed": len(suggestions),
        "suggestions_by_group": suggestions,
        "b_loss_cases_detail": b_loss_cases
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 개선사항 보고서 저장 완료: {output_path}")
    return report


# 보고서 저장
improvement_report = save_improvement_report(
    improvement_suggestions,
    b_loss_or_tie_cases,
    "260122_이미지프롬프트_개선사항_보고서.json"
)

# 종합 요약 출력
print(f"\n{'='*120}")
print("📋 이미지 프롬프트 템플릿 개선사항 종합 요약")
print(f"{'='*120}")

priority_high = [g for g, s in improvement_suggestions.items() if s.get('priority') == '높음']
priority_medium = [g for g, s in improvement_suggestions.items() if s.get('priority') == '중간']
priority_low = [g for g, s in improvement_suggestions.items() if s.get('priority') == '낮음']

print(f"\n🔴 높은 우선순위 ({len(priority_high)}개 그룹): {priority_high}")
print(f"🟡 중간 우선순위 ({len(priority_medium)}개 그룹): {priority_medium}")
print(f"🟢 낮은 우선순위 ({len(priority_low)}개 그룹): {priority_low}")

print(f"\n📝 그룹별 핵심 개선사항:")
for group, suggestion in improvement_suggestions.items():
    if "error" not in suggestion:
        print(f"\n  [{group}]")
        print(f"  {suggestion.get('summary', 'N/A')}")


✅ 개선사항 보고서 저장 완료: 260122_이미지프롬프트_개선사항_보고서.json

📋 이미지 프롬프트 템플릿 개선사항 종합 요약

🔴 높은 우선순위 (2개 그룹): ['A-3', 'B-3']
🟡 중간 우선순위 (0개 그룹): []
🟢 낮은 우선순위 (0개 그룹): []

📝 그룹별 핵심 개선사항:

  [A-3]
  시각 디자인업에 맞는 창의적이고 역동적인 작업 환경을 강조하기 위해, 관련 키워드와 조명, 구도, 색상 팔레트를 조정합니다. 업종 특화성을 높이기 위해 프롬프트 템플릿을 개선합니다.

  [B-3]
  B-3 그룹의 광고 이미지를 개선하기 위해 전체적인 아케이드 경험과 활기찬 분위기를 강조하는 방향으로 설정을 변경합니다. 이를 위해 네온 조명과 넓은 구도를 사용하며, 복고풍의 활기찬 분위기를 담을 수 있는 키워드를 추가합니다.
